# How to use Awkward Arrays in C++ with cppyy

:::{warning}

Awkward Array can only work with `cppyy` 3.0.1 or later. As of writing, this is _only_ available from Git.
:::

:::{warning}
`cppyy` must be in a different venv or conda environment from ROOT, if you have installed ROOT.
:::

The [cppyy](https://cppyy.readthedocs.io/en/latest/index.html) is an automatic, run-time, Python-C++ bindings generator, for calling C++ from Python and Python from C++. `cppyy` is based on the C++ interpreter `Cling`.

`cppyy` can understand Awkward Arrays. When an {class}`ak.Array` type is passed to a C++ function defined in `cppyy`, a `__cast_cpp__` magic function of an {class}`ak.Array` is invoked. The function dynamically generates a C++ type and a view of the array, if it has not been generated yet.

The view is a lightweight 40-byte C++ object dynamically allocated on the stack. This view is generated on demand - and only once per Awkward Array, the data are not copied.

In [1]:
import awkward as ak
ak.__version__

'2.1.1'

In [2]:
import awkward._connect.cling

In [3]:
import cppyy
cppyy.__version__

'3.0.0'

Let's define an Awkward Array as a list of records:

In [4]:
array = ak.Array(
    [
        [{"x": 1, "y": [1.1]}, {"x": 2, "y": [2.2, 0.2]}],
        [],
        [{"x": 3, "y": [3.0, 0.3, 3.3]}],
    ]
)
array

<Array [[{x: 1, y: [1.1]}, {...}], ...] type='3 * var * {x: int64, y: var *...'>

This example shows a templated C++ function that takes an Awkward Array and iterates over the list of records:

In [5]:
source_code = """
template<typename T>
double go_fast_cpp(T& awkward_array) {
    double out = 0.0;

    for (auto list : awkward_array) {
        for (auto record : list) {
            for (auto item : record.y()) {
                out += item;
            }
        }
    }

    return out;
}
"""

cppyy.cppdef(source_code)

True

The C++ type of an Awkward Array is a made-up type;
`awkward::ListArray_hyKwTH3lk1A`.

In [6]:
array.cpp_type

'awkward::ListArray_hyKwTH3lk1A'

Awkward Arrays are dynamically typed, so in a C++ context, the type name is hashed. In practice, there is no need to know the type. The C++ code should use a placeholder type specifier `auto`. The type of the variable that is being declared will be automatically deduced from its initializer.

In a Python contexts, when a templated function requires a C++ type as a Python string, it can use the `ak.Array.cpp_type` property:

In [7]:
out = cppyy.gbl.go_fast_cpp[array.cpp_type](array)

In [8]:
%%timeit

out = cppyy.gbl.go_fast_cpp[array.cpp_type](array)

3.93 µs ± 135 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [9]:
%%timeit

ak.sum(array["y"])

184 µs ± 15.9 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


But the result is the same.

In [10]:
assert out == ak.sum(array["y"])